In [3]:
%load_ext Cython

In [3]:
%%cython --annotate

from cython cimport bytearray
ctypedef unsigned char uint8_t
cdef bytearray_setval(uint8_t[::1] a, int val):
    for i in range(len(a)):
        a[i] = val

cdef uint8_t[::1] a = bytearray(3)
bytearray_setval(a, 2)
cdef uint8_t[::1] b = bytearray(5)
bytearray_setval(b, 3)
print(bytearray(a) + bytearray(b))

Content of stdout:
_cython_magic_c6c4af011ab88bb3adafdd5c00519db7462d69e5.c
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_c6c4af011ab88bb3adafdd5c00519db7462d69e5.c(1412): warning C4005: '__pyx_nonatomic_int_type'ÿ: red‚finition de macro
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_c6c4af011ab88bb3adafdd5c00519db7462d69e5.c(1363): note: voir la d‚finition pr‚c‚dente de '__pyx_nonatomic_int_type'
   Cr‚ation de la bibliothŠque C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_c6c4af011ab88bb3adafdd5c00519db7462d69e5.cp310-win_amd64.lib et de l'objet C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_c6c4af011ab88bb3adafdd5c00519db7462d69e5.cp310-win_amd64.exp
G‚n‚ration de code en cours
Fin de la g‚n‚ration du codebytearray(b'\x02\x02\x02\x03\x03\x03\x03\x03')


In [26]:
%%cython --annotate

from cython.view cimport array as cvarray
import time

import numpy as np
cimport numpy as np

from cpython cimport array
from libc.stdlib cimport malloc, free

cdef double* ptr

cdef long N = 1000000
cdef array.array Lrange = array.array('i', [1,10,100,1000,10000])
cdef long i
cdef int L

for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        a = cvarray((L,),sizeof(double),'d')
    t2 = time.process_time()
    print('Elapsed time to make cython array with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        a = np.arange(L)
    t2 = time.process_time()
    print('Elapsed time to make numpy array with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        ptr = <double*> malloc(sizeof(double) * L)
        free(ptr)
    t2 = time.process_time()
    print('Elapsed time with raw c-allocation with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

cdef array.array ar, template = array.array('d')
for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        ar = array.clone(template, L, False)
    t2 = time.process_time()
    print('Elapsed time to make cpython array with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')



#TODO: C array https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html#quickstart
#for L in [1,10,100,1000,10000]:
#    t1 = time.process_time()
#    for i in range(N):
#        cdef double[L] arc
#    t2 = time.process_time()
#    print('Elapsed time to make c array with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

Content of stdout:
_cython_magic_ddf18ad79e6689df9f858553ec538a23e234430a.c
C:\Users\THEMATRIXNEO\miniconda3\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_ddf18ad79e6689df9f858553ec538a23e234430a.c(1493): warning C4005: '__pyx_nonatomic_int_type'ÿ: red‚finition de macro
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_ddf18ad79e6689df9f858553ec538a23e234430a.c(1444): note: voir la d‚finition pr‚c‚dente de '__pyx_nonatomic_int_type'
   Cr‚ation de la bibliothŠque C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_ddf18ad79e6689df9f858553ec538a23e234430a.cp310-win_amd64.lib et de l'objet C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_ddf18ad79e6689df9f858553ec538a23e234430a.cp310-win_amd64.exp
G‚n‚ration de code en cours
Fin d

In [24]:
%%cython --annotate

import time
from cython cimport bytearray
from cpython cimport array

cdef long N = 1000000
cdef array.array Lrange = array.array('i', [1,10,100,1000,10000])
cdef long i
cdef int L

cdef bytearray barr
for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        barr = bytearray(L)
    t2 = time.process_time()
    print('Elapsed time to make bytearray with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

cdef unsigned char[::1] mbarr
for L in Lrange:
    t1 = time.process_time()
    for i in range(N):
        mbarr = bytearray(L)
    t2 = time.process_time()
    print('Elapsed time to make bytearray memoryview with length of ' +str(L)+' is '+str((t2-t1)/N*1e6)+' us')

In [4]:
%%cython --annotate

# cython: language_level=3
# cython: boundscheck=False
# cython: wraparound=False

import time
import sys

from cpython.array cimport array, clone
from cython.view cimport array as cvarray
from libc.stdlib cimport malloc, free
import numpy as numpy
cimport numpy as numpy

cdef int loops

def timefunc(name):
    def timedecorator(f):
        cdef int L, i

        print("Running", name)
        for L in [1, 10, 100, 1000, 10000, 100000]:
            start = time.process_time()
            f(L)
            end = time.process_time()
            print(format((end-start) / loops * 1e6, "2f"), end=" ")
            sys.stdout.flush()

        print("μs")
    return timedecorator

print()
print("INITIALISATIONS")
loops = 100000

@timefunc("cpython.array buffer")
def _(int L):
    cdef int i
    cdef array[double] arr, template = array('d')

    for i in range(loops):
        arr = clone(template, L, False)

    # Prevents dead code elimination
    str(arr[0])

@timefunc("cpython.array memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr
    cdef array template = array('d')

    for i in range(loops):
        arr = clone(template, L, False)

    # Prevents dead code elimination
    str(arr[0])

@timefunc("cpython.array raw C type")
def _(int L):
    cdef int i
    cdef array arr, template = array('d')

    for i in range(loops):
        arr = clone(template, L, False)

    # Prevents dead code elimination
    str(arr[0])

@timefunc("numpy.empty_like memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr
    template = numpy.empty((L,), dtype='double')

    for i in range(loops):
        arr = numpy.empty_like(template)

    # Prevents dead code elimination
    str(arr[0])

@timefunc("malloc")
def _(int L):
    cdef int i
    cdef double* arrptr

    for i in range(loops):
        arrptr = <double*> malloc(sizeof(double) * L)
        free(arrptr)

    # Prevents dead code elimination
    str(arrptr[0])

@timefunc("malloc memoryview")
def _(int L):
    cdef int i
    cdef double* arrptr
    cdef double[::1] arr

    for i in range(loops):
        arrptr = <double*> malloc(sizeof(double) * L)
        arr = <double[:L]>arrptr
        free(arrptr)

    # Prevents dead code elimination
    str(arr[0])

@timefunc("cvarray memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr

    for i in range(loops):
        arr = cvarray((L,),sizeof(double),'d')

    # Prevents dead code elimination
    str(arr[0])



print()
print("ITERATING")
loops = 1000

@timefunc("cpython.array buffer")
def _(int L):
    cdef int i
    cdef array[double] arr = clone(array('d'), L, False)

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    # Prevents dead-code elimination
    str(d)

@timefunc("cpython.array memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr = clone(array('d'), L, False)

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    # Prevents dead-code elimination
    str(d)

@timefunc("cpython.array raw C type")
def _(int L):
    cdef int i
    cdef array arr = clone(array('d'), L, False)

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    # Prevents dead-code elimination
    str(d)

@timefunc("numpy.empty_like memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr = numpy.empty((L,), dtype='double')

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    # Prevents dead-code elimination
    str(d)

@timefunc("malloc")
def _(int L):
    cdef int i
    cdef double* arrptr = <double*> malloc(sizeof(double) * L)

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arrptr[i]

    free(arrptr)

    # Prevents dead-code elimination
    str(d)

@timefunc("malloc memoryview")
def _(int L):
    cdef int i
    cdef double* arrptr = <double*> malloc(sizeof(double) * L)
    cdef double[::1] arr = <double[:L]>arrptr

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    free(arrptr)

    # Prevents dead-code elimination
    str(d)

@timefunc("cvarray memoryview")
def _(int L):
    cdef int i
    cdef double[::1] arr = cvarray((L,),sizeof(double),'d')

    cdef double d
    for i in range(loops):
        for i in range(L):
            d = arr[i]

    # Prevents dead-code elimination
    str(d)


Content of stdout:
_cython_magic_fdf74a4c94f4ab3f73cfe617fa5fc1ad8e8c6a1c.c
C:\Users\THEMATRIXNEO\miniconda3\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_fdf74a4c94f4ab3f73cfe617fa5fc1ad8e8c6a1c.c(1465): warning C4005: '__pyx_nonatomic_int_type'ÿ: red‚finition de macro
C:\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_fdf74a4c94f4ab3f73cfe617fa5fc1ad8e8c6a1c.c(1416): note: voir la d‚finition pr‚c‚dente de '__pyx_nonatomic_int_type'
   Cr‚ation de la bibliothŠque C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_fdf74a4c94f4ab3f73cfe617fa5fc1ad8e8c6a1c.cp310-win_amd64.lib et de l'objet C:\Users\THEMATRIXNEO\.ipython\cython\Users\THEMATRIXNEO\.ipython\cython\_cython_magic_fdf74a4c94f4ab3f73cfe617fa5fc1ad8e8c6a1c.cp310-win_amd64.exp
G‚n‚ration de code en cours
Fin d